In [2]:
!pip install google-genai
#

In [5]:
# Create a Python function that uses Gemini to classify user questions into one of the following
# categories: Employment, General Information, Emergency Services, or Tax Related

# Using an LLM for classification or sentiment analysis are examples

import os
from google import genai
from google.genai import types

PROJECT_ID = "qwiklabs-gcp-01-c75658565206"
LOCATION = "us-west1"
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

def classify_user_question(prompt: str) -> str:
  """
  Uses the Gemini model to classify a user question into one of four categories:
  Employment, General Information, Emergency Services, or Tax Related.

  Args:
    prompt: The user's question as a string.

  Returns:
    The classified category as a string (e.g., 'Employment').
  """

  # Select a suitable model for fast classification (e.g., gemini-2.5-flash)
  model = 'gemini-2.5-flash'

  # The system instruction guides the model's behavior and output format.
  system_instruction = (
      "You are an expert classification system. "
      "Your sole task is to classify the user's message into one of the "
      "following four categories: 'Employment', 'General Information', "
      "'Emergency Services', or 'Tax Related'. "
      "You MUST output ONLY the name of the category."
  )

  response = client.models.generate_content(
      model=model,
      contents=f"Message: {prompt}",
      config=types.Generate  ContentConfig(
          system_instruction=system_instruction,
          # Setting temperature to 0.0 encourages deterministic and strict classification
          temperature=0.0
      ),
  )

  # Clean up the response text and return the category
  return response.text.strip()


# --- Example Usage ---
print(classify_user_question("I need to know my W-2 status."))
print(classify_user_question("What is the capital of France?"))
print(classify_user_question("Is there a fire nearby?"))
print(classify_user_question("How do I apply for the open position?"))

Tax Related
General Information
Emergency Services
Employment


In [7]:
# Create a second function that generates social media posts for government announcements like
# weather emergencies, holidays, school closings, etc.


def generate_government_post(prompt: str) -> str:
  """
  Generates a concise, authoritative social media announcement (e.g., Tweet)
  for public service purposes based on the user's request.

  Args:
    prompt: The core information for the announcement (e.g., "All schools
            are closed due to snow today").

  Returns:
    The generated, formatted social media post.
  """
  # Using a fast model for text generation
  model = 'gemini-2.5-flash'

  # The system instruction defines the persona and rules the model must follow
  system_instruction = (
      "You are a public communications officer writing critical announcements "
      "for social media (like Twitter/X). Your tone must be authoritative, "
      "clear, and concise. All posts must be highly readable and actionable."
  )

  # Use few-shot prompting to guide the model on formatting and rules
  few_shot_prompt = (
      """
      # Rules:
      1. Max character limit is 240.
      2. Use ALL CAPS for critical information.
      3. Include relevant hashtags like #SafetyAlert, #Weather, or #Update.

      # Example Input:
      City Hall will be closed for the holiday tomorrow.

      # Example Output:
      REMINDER: City Hall and all non-essential municipal services will be CLOSED tomorrow, Dec 25, for the holiday. Normal operations resume Wednesday. #HolidayUpdate

      # Input:
      {0}

      # Output:
      """.format(prompt)
  )

  response = client.models.generate_content(
      model=model,
      contents=few_shot_prompt,
      config=types.GenerateContentConfig(
          system_instruction=system_instruction,
          temperature=0.7 # Allow for some creativity in phrasing
      ),
  )

  return response.text.strip()


# --- Example of how to use the function in your notebook ---
print("--- Weather Emergency Post ---")
print(generate_government_post("A severe thunderstorm warning is in effect until 8 PM. Seek shelter immediately."))

print("\n--- School Closing Post ---")
print(generate_government_post("All district schools are closed today, Tuesday, due to a power outage."))

--- Weather Emergency Post ---
SEVERE THUNDERSTORM WARNING IN EFFECT UNTIL 8 PM. SEEK SHELTER IMMEDIATELY. Stay indoors, away from windows. #WeatherAlert #SafetyFirst

--- School Closing Post ---
ALERT: ALL DISTRICT SCHOOLS ARE CLOSED TODAY, TUESDAY, due to a POWER OUTAGE. NO CLASSES WILL BE HELD. #SchoolClosure #PowerOutage #SafetyAlert


In [ ]:
# Write unit tests for each function using pytest.

In [ ]:
# Use the Google Evaluation API to evaluate and compare Gemini responses from different
# prompts.